<a href="https://colab.research.google.com/github/GoldPapaya/info256-applied-nlp/blob/main/7.lm/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp25/blob/main/7.lm/BERT.ipynb)

In this notebook, we'll explore the basics of token representations in BERT and use it to find token nearest neighbors.  You should open this notebook in Google Colab.


In [1]:
from transformers import BertModel, BertTokenizer
import numpy as np

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT uses WordPiece tokenization, which breaks down words that don't appear within its 30K-word vocabulary into small pieces.  The word "Ishmael", for instanced, is tokenized as ["is", "##hma", "##el"]

In [3]:
inputs = tokenizer("Call me Ishmael", return_tensors="pt")
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

['[CLS]', 'call', 'me', 'is', '##hma', '##el', '[SEP]']

In [4]:
inputs = tokenizer("BERT is supercalifragilisticexpialidocious", return_tensors="pt")
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

['[CLS]',
 'bert',
 'is',
 'super',
 '##cal',
 '##if',
 '##rag',
 '##ilis',
 '##tic',
 '##ex',
 '##pia',
 '##lid',
 '##oc',
 '##ious',
 '[SEP]']

As mentioned in class, notice how every sentence input to BERT is wrapped in two tags: a start [CLS] tag and an ending [SEP] tag.  BERT will generate representations of each WordPiece token, including these special [CLS] and [SEP] tags.

To generate representations for the input tokens, simply pass the input through the BERT model:

In [5]:
inputs = tokenizer("This jam is delicious", return_tensors="pt")
outputs = model(**inputs)
tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

['[CLS]', 'this', 'jam', 'is', 'delicious', '[SEP]']

Representations for each of BERT layers (12 in this model) are accessible here, but let's explore just the outputs from the final layer.  This BERT model has 768-dimensional representations, so this 6-token input ([CLS, this, jam, is, delicious, [SEP]) has an output that is is a 1 x 6 tokens x 768 dimensional tensor.

In [6]:
last_hidden_states = outputs.last_hidden_state

In [7]:
print(outputs.last_hidden_state.shape)

torch.Size([1, 6, 768])


What can we do with just these representations?  While we used word2vec-style static embeddings to find nearest neighbors for word *types*, we can do the same here for word *tokens*.

In [8]:
def cosine_similarity(a, b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [9]:
query = "I ate some jam with toast"

In [17]:
comp_sents = ["She got me out of a real jam", "This jam is made of strawberries", "I sat in a traffic jam for 2 hours", "The Grateful Dead used to jam for like two days straight.", "My grandma makes the best jam.", "I had to jam on the brakes to avoid hitting him."]

In [11]:
def get_bert_for_token(string, term):

    # tokenize
    inputs = tokenizer(string, return_tensors="pt")

    # convert input ids to words
    tokens=tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # find the first location of the query term among those tokens (so we know which BERT rep to use)
    term_idx=tokens.index(term)

    outputs = model(**inputs)

    # return the BERT rep for that token index
    # The output is a pytorch tensor object, but let's convert it to a numpy object to work with numpy functions

    return outputs.last_hidden_state[0][term_idx].detach().numpy()



In [16]:
query_rep=get_bert_for_token(query, "jam")
print(query_rep.shape)

(768,)


In [18]:
vals=[]
for sent in comp_sents:
    comp_rep = get_bert_for_token(sent, "jam")
    cos_sim = cosine_similarity(query_rep, comp_rep)
    vals.append((cos_sim, query, sent))

for c, q, s in reversed(sorted(vals)):
    print("%.3f\t%s\t%s" % (c, q, s))

0.736	I ate some jam with toast	My grandma makes the best jam.
0.665	I ate some jam with toast	This jam is made of strawberries
0.480	I ate some jam with toast	I sat in a traffic jam for 2 hours
0.443	I ate some jam with toast	The Grateful Dead used to jam for like two days straight.
0.385	I ate some jam with toast	She got me out of a real jam
0.290	I ate some jam with toast	I had to jam on the brakes to avoid hitting him.


**Q**: Brainstorm the variety of things you can do with token-level word representations like this.  As one example, adapt the code above to find *any* word that is most similar to *jam* in "I ate some jam with toast" in the following sentences.  Are you able to find token-level synonyms this way?

In [14]:
comp_sents = ["My grandma makes the best jelly.", "Jelly is made of strawberries"]